<a href="https://colab.research.google.com/github/hoky1227/Transformer_based-recommendation/blob/main/Transformer_with_ECF_Algorithm_1230_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Module 가져오기

In [3]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import math
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean

import warnings 

warnings.filterwarnings('ignore')

### Data Road

In [4]:
TFoutputs = pd.read_csv(path + "Transformer_test_outputs_20.csv")
TFoutputs.pop('Unnamed: 0')
TFoutputs

,train,label,split_train_token,split_label_token,predict_item
0,ENAMEL FLOWER JUG CREAM&&DOOR MAT FAIRY CAKE&&...,RED SPOTTY TABLECLOTH,<esp> ENAMEL FLOWER JUG CREAM <esp> DOOR MAT F...,<esp> RED SPOTTY TABLECLOTH <esp>,<esp> ENAMEL BREAD BIN CREAM <esp>
1,GAOLERS KEYS DECORATIVE GARDEN&&HAND OPEN SHAP...,SET OF 2 FANCY FONT TEA TOWELS,<esp> GAOLERS KEYS DECORATIVE GARDEN <esp> HAN...,<esp> SET OF 2 FANCY FONT TEA TOWELS <esp>,<esp> PAPER BUNTING RETRO SPOTS <esp>
2,SET OF 9 HEART SHAPED BALLOONS&&SET OF 9 BLACK...,CERAMIC CHERRY CAKE MONEY BANK,<esp> SET OF 9 HEART SHAPED BALLOONS <esp> SET...,<esp> CERAMIC CHERRY CAKE MONEY BANK <esp>,<esp> ASSTD DESIGN BUBBLE GUM RING <esp>
3,HANGING HEART ZINC T-LIGHT HOLDER&&OPULENT VEL...,COLOUR GLASS. STAR T-LIGHT HOLDER,<esp> HANGING HEART ZINC T-LIGHT HOLDER <esp> ...,<esp> COLOUR GLASS. STAR T-LIGHT HOLDER <esp>,<esp> HANGING HEART ZINC T-LIGHT HOLDER <esp>
4,BAKING SET 9 PIECE RETROSPOT&&PARTY INVITES WO...,PARTY INVITES FOOTBALL,<esp> BAKING SET 9 PIECE RETROSPOT <esp> PARTY...,<esp> PARTY INVITES FOOTBALL <esp>,<esp> PARTY INVITES WOODLAND <esp>
...,...,...,...,...,...
14236,ENAMEL MEASURING JUG CREAM&&CREAM HEART CARD H...,IVORY KITCHEN SCALES,<esp> ENAMEL MEASURING JUG CREAM <esp> CREAM H...,<esp> IVORY KITCHEN SCALES <esp>,<esp> DOOR MAT UNION FLAG <esp>
14237,TOILET METAL SIGN&&SAVE THE PLANET MUG&&KINGS ...,POTTERING IN THE SHED METAL SIGN,<esp> TOILET METAL SIGN <esp> SAVE THE PLANET ...,<esp> POTTERING IN THE SHED METAL SIGN <esp>,<esp> BATHROOM METAL SIGN <esp>
14238,IVORY HANGING DECORATION HEART&&HEART IVORY T...,COLOUR GLASS T-LIGHT HOLDER HANGING,<esp> IVORY HANGING DECORATION HEART <esp> HE...,<esp> COLOUR GLASS T-LIGHT HOLDER HANGING <esp>,<esp> HEART T-LIGHT HOLDER <esp>
14239,LAVENDER SCENTED FABRIC HEART&&PLACE SETTING W...,MURANO STYLE GLASS BRACELET PINK,<esp> LAVENDER SCENTED FABRIC HEART <esp> PLAC...,<esp> MURANO STYLE GLASS BRACELET PINK <esp>,<esp> RED GINGHAM ROSE JEWELLERY BOX <esp>


In [5]:
checker = pd.Series([i in list(TFoutputs['split_label_token']) for i in TFoutputs['predict_item']])
# filter = checker.replace(False, np.NaN)
print(checker.value_counts())
filter = checker[checker == False].index
len(filter)

True     14040
False      201
dtype: int64


201

In [6]:
TFoutputs['filter'] = checker
# falses = TFoutputs.isna().sum()
# TFoutputs.dropna(inplace=True)
# TFoutputs.reset_index(drop=True, inplace=True)
# TFoutputs

In [7]:
TFoutputs

,train,label,split_train_token,split_label_token,predict_item,filter
0,ENAMEL FLOWER JUG CREAM&&DOOR MAT FAIRY CAKE&&...,RED SPOTTY TABLECLOTH,<esp> ENAMEL FLOWER JUG CREAM <esp> DOOR MAT F...,<esp> RED SPOTTY TABLECLOTH <esp>,<esp> ENAMEL BREAD BIN CREAM <esp>,True
1,GAOLERS KEYS DECORATIVE GARDEN&&HAND OPEN SHAP...,SET OF 2 FANCY FONT TEA TOWELS,<esp> GAOLERS KEYS DECORATIVE GARDEN <esp> HAN...,<esp> SET OF 2 FANCY FONT TEA TOWELS <esp>,<esp> PAPER BUNTING RETRO SPOTS <esp>,True
2,SET OF 9 HEART SHAPED BALLOONS&&SET OF 9 BLACK...,CERAMIC CHERRY CAKE MONEY BANK,<esp> SET OF 9 HEART SHAPED BALLOONS <esp> SET...,<esp> CERAMIC CHERRY CAKE MONEY BANK <esp>,<esp> ASSTD DESIGN BUBBLE GUM RING <esp>,True
3,HANGING HEART ZINC T-LIGHT HOLDER&&OPULENT VEL...,COLOUR GLASS. STAR T-LIGHT HOLDER,<esp> HANGING HEART ZINC T-LIGHT HOLDER <esp> ...,<esp> COLOUR GLASS. STAR T-LIGHT HOLDER <esp>,<esp> HANGING HEART ZINC T-LIGHT HOLDER <esp>,True
4,BAKING SET 9 PIECE RETROSPOT&&PARTY INVITES WO...,PARTY INVITES FOOTBALL,<esp> BAKING SET 9 PIECE RETROSPOT <esp> PARTY...,<esp> PARTY INVITES FOOTBALL <esp>,<esp> PARTY INVITES WOODLAND <esp>,True
...,...,...,...,...,...,...
14236,ENAMEL MEASURING JUG CREAM&&CREAM HEART CARD H...,IVORY KITCHEN SCALES,<esp> ENAMEL MEASURING JUG CREAM <esp> CREAM H...,<esp> IVORY KITCHEN SCALES <esp>,<esp> DOOR MAT UNION FLAG <esp>,True
14237,TOILET METAL SIGN&&SAVE THE PLANET MUG&&KINGS ...,POTTERING IN THE SHED METAL SIGN,<esp> TOILET METAL SIGN <esp> SAVE THE PLANET ...,<esp> POTTERING IN THE SHED METAL SIGN <esp>,<esp> BATHROOM METAL SIGN <esp>,True
14238,IVORY HANGING DECORATION HEART&&HEART IVORY T...,COLOUR GLASS T-LIGHT HOLDER HANGING,<esp> IVORY HANGING DECORATION HEART <esp> HE...,<esp> COLOUR GLASS T-LIGHT HOLDER HANGING <esp>,<esp> HEART T-LIGHT HOLDER <esp>,True
14239,LAVENDER SCENTED FABRIC HEART&&PLACE SETTING W...,MURANO STYLE GLASS BRACELET PINK,<esp> LAVENDER SCENTED FABRIC HEART <esp> PLAC...,<esp> MURANO STYLE GLASS BRACELET PINK <esp>,<esp> RED GINGHAM ROSE JEWELLERY BOX <esp>,True


In [8]:
pred = TFoutputs['predict_item']
pred = pred.str.replace('<esp>', '')
pred = pred.str.strip()
pred.head()

0               ENAMEL BREAD BIN CREAM
1            PAPER BUNTING RETRO SPOTS
2         ASSTD DESIGN BUBBLE GUM RING
3    HANGING HEART ZINC T-LIGHT HOLDER
4               PARTY INVITES WOODLAND
Name: predict_item, dtype: object

In [9]:
# falses = falses['filter']
# falses

falses = pd.Series([TFoutputs['predict_item'][i] for i in filter])
falses = falses.str.replace('<esp>', '')
falses = falses.str.strip()
print(len(falses))
falses.head()

201


0       BATHROOM SCALES, TROPICAL BEACH
1      ANT COPPER RED BOUDICCA BRACELET
2              PINK PATCH CUSHION COVER
3    SMALL CAMPHOR WOOD FIELD  MUSHROOM
4    CHOCOLATE 3 WICK MORRIS BOX CANDLE
dtype: object

In [10]:
filter[:5]

Int64Index([34, 46, 277, 370, 399], dtype='int64')

In [11]:
df = pd.read_csv(path + "data_customer_based.csv")
df.pop('Unnamed: 0')
df

,Customer,Detail,label
0,32346.0,RED SPOTTY CHILDS UMBRELLA&&EDWARDIAN PARASOL ...,DOORMAT UNION JACK GUNS AND ROSES
1,32349.0,WRAP PINK FAIRY CAKES&&VINTAGE CARAVAN GIFT WR...,LOVE LARGE WOOD LETTERS
2,32355.0,TOOTHPASTE TUBE PEN&&KIDS RAIN MAC BLUE&&KIDS ...,CERAMIC CAKE STAND + HANGING CAKES
3,32358.0,ASSORTED CHEESE FRIDGE MAGNETS&&ASSORTED ICE C...,JUMBO SHOPPER VINTAGE RED PAISLEY
4,32359.0,TEA BAG PLATE RED SPOTTY&&FAIRY SOAP SOAP HOLD...,RED KITCHEN SCALES
...,...,...,...
2672,38276.0,60 TEATIME FAIRY CAKE CASES&&PACK OF 72 SKULL ...,VICTORIAN SEWING BOX LARGE
2673,38281.0,"SPACEBOY BIRTHDAY CARD&&ELEPHANT, BIRTHDAY CAR...",AIRLINE BAG VINTAGE JET SET WHITE
2674,38283.0,RETRO SPOT MUG&&PACK OF 6 SKULL PAPER PLATES&&...,VINTAGE UNION JACK BUNTING
2675,38285.0,PAPER BUNTING WHITE LACE&&RIDGED GLASS T-LIGHT...,GLASS CAKE STAND MIRRORED BASE


In [12]:
user2item = df.copy()

In [13]:
user2item.columns = ['유저 식별키', '유저 구매 상품 식별키 묶음', '라벨']
user2item

,유저 식별키,유저 구매 상품 식별키 묶음,라벨
0,32346.0,RED SPOTTY CHILDS UMBRELLA&&EDWARDIAN PARASOL ...,DOORMAT UNION JACK GUNS AND ROSES
1,32349.0,WRAP PINK FAIRY CAKES&&VINTAGE CARAVAN GIFT WR...,LOVE LARGE WOOD LETTERS
2,32355.0,TOOTHPASTE TUBE PEN&&KIDS RAIN MAC BLUE&&KIDS ...,CERAMIC CAKE STAND + HANGING CAKES
3,32358.0,ASSORTED CHEESE FRIDGE MAGNETS&&ASSORTED ICE C...,JUMBO SHOPPER VINTAGE RED PAISLEY
4,32359.0,TEA BAG PLATE RED SPOTTY&&FAIRY SOAP SOAP HOLD...,RED KITCHEN SCALES
...,...,...,...
2672,38276.0,60 TEATIME FAIRY CAKE CASES&&PACK OF 72 SKULL ...,VICTORIAN SEWING BOX LARGE
2673,38281.0,"SPACEBOY BIRTHDAY CARD&&ELEPHANT, BIRTHDAY CAR...",AIRLINE BAG VINTAGE JET SET WHITE
2674,38283.0,RETRO SPOT MUG&&PACK OF 6 SKULL PAPER PLATES&&...,VINTAGE UNION JACK BUNTING
2675,38285.0,PAPER BUNTING WHITE LACE&&RIDGED GLASS T-LIGHT...,GLASS CAKE STAND MIRRORED BASE


# Product2Vector 생성

### Jaccard 유사도

In [14]:
detail = pd.read_csv(path + "Detail.csv")
detail.pop('Unnamed: 0')
print(len(detail))
detail = detail['0'].to_list()
len(detail)

3487


3487

In [15]:
# pd.Series([i in detail for i in falses]).value_counts()

In [16]:
def JaccardSimilarity(inp1, inp2):
    list_inp1 = inp1.split()
    list_inp2 = inp2.split()
    mom = set(list_inp1).union(set(list_inp2))
    son = set(list_inp1).intersection(set(list_inp2))
    # print(mom)
    # print(son)
    return len(son)/len(mom)

In [17]:
# detail = TFoutputs['label'].unique().tolist()
# print(detail[-1])
# len(detail)

In [18]:
pred[75]

'SET OF 72 GREEN PAPER DOILIES'

In [19]:
pd.Series([i in detail for i in pred]).value_counts()

True     14227
False       14
dtype: int64

In [20]:
asdf = []

for v in falses:
    max = 0
    for d in detail:
        score = JaccardSimilarity(v, d)
        if max < score:
            max = score
            temp = d
    if max == 0: asdf.append(v)
    id = pred[pred == v].index
    for i in id:
        pred[i] = temp

In [21]:
len(asdf)

0

In [22]:
falses[0]

'BATHROOM SCALES, TROPICAL BEACH'

In [23]:
pred[75]

'SET OF 72 GREEN PAPER DOILIES'

In [24]:
pd.Series([i in detail for i in pred]).value_counts()

True    14241
dtype: int64

In [25]:
changes = pd.DataFrame(columns=['Original', 'Changed'])
changes['Original'] = falses
changes['Changed'] = pred[filter].to_list()
changes

,Original,Changed
0,"BATHROOM SCALES, TROPICAL BEACH","BATHROOM SCALES, TROPICAL BEACH"
1,ANT COPPER RED BOUDICCA BRACELET,ANT COPPER RED BOUDICCA BRACELET
2,PINK PATCH CUSHION COVER,PINK YELLOW PATCH CUSHION COVER
3,SMALL CAMPHOR WOOD FIELD MUSHROOM,SMALL CAMPHOR WOOD FIELD MUSHROOM
4,CHOCOLATE 3 WICK MORRIS BOX CANDLE,CHOCOLATE 3 WICK MORRIS BOX CANDLE
...,...,...
196,CHOCOLATE 3 WICK MORRIS BOX CANDLE,CHOCOLATE 3 WICK MORRIS BOX CANDLE
197,WHITE/PINK MINI CRYSTALS NECKLACE,WHITE/PINK MINI CRYSTALS NECKLACE
198,BLACK PILLAR CANDLE SILVER FLOCK,BLACK PILLAR CANDLE SILVER FLOCK
199,TRADITIONAL CHRISTMAS RIBBONS,TRADITIONAL CHRISTMAS RIBBONS


In [26]:
changes.to_csv(path + 'changed_products.csv', mode='w')

### Word2vec Model 학습

In [27]:
model = Word2Vec([item_list.split('&&') for item_list in user2item['유저 구매 상품 식별키 묶음']], size=100, window=5, min_count=1, negative=15, iter=20, sample=0.00001, sg=1)

In [28]:
# 유저 성향 벡터 구하기 
user_propensity_vector = []
for item_list in tqdm_notebook(user2item['유저 구매 상품 식별키 묶음']):
    item_list = item_list.split('&&')
    bbb = np.array(np.ones(100))
    for aaa in item_list:
        bbb *= model[aaa]
        bbb = bbb/norm(bbb)
    user_propensity_vector.append(bbb)

In [29]:
# 유사 product 찾기
# def find_item(user, n=10):
#     item_code = user2item[user2item['유저 식별키']==user]['유저 구매 상품 식별키 묶음'].values[0].split('&&')[-1]
#     return list(dict(model.wv.most_similar(item_code, topn=n)))

def find_item(item, n=10):
    return list(dict(model.wv.most_similar(item, topn=n)))

# 유저 유사도 계산 및 유사 유저 찾기
def user_sim(A, B): 
    A_vec = user_propensity_vector[user2item[user2item['유저 식별키']==A].index[0]]
    B_vec = user_propensity_vector[user2item[user2item['유저 식별키']==B].index[0]]
    return euclidean(A_vec.reshape(-1, 1), B_vec.reshape(-1, 1))

def find_user(user):
    user_index = user2item[user2item['유저 식별키']==user].index[0]
    min_user_similar = min(user_matrix_f[user_index].drop(user_index))
    return user2item.iloc[{j:i for i, j in enumerate(user_matrix_f[user_index])}[min_user_similar]]['유저 식별키']

In [30]:
# # 유저 유사도 매트릭스
# user_matrix_f = pd.DataFrame()
# for i in tqdm_notebook(range(len(user_propensity_vector))):
#     tmp = []
#     for j in range(len(user_propensity_vector)):
#         tmp.append(euclidean(user_propensity_vector[i], user_propensity_vector[j]))
#     user_matrix_f = pd.concat([user_matrix_f, pd.DataFrame(tmp)], axis = 1)
# user_matrix_f.columns = user_matrix_f.index
# user_matrix_f

In [31]:
find_item(pred[0])

['TEA COSY VINTAGE UNION JACK',
 'ENAMEL MEASURING JUG CREAM',
 'ENAMEL WATERING CAN CREAM',
 'ENAMEL FLOWER JUG CREAM',
 'ENAMEL WASH BOWL CREAM',
 'ENAMEL COLANDER CREAM',
 'MINT KITCHEN SCALES',
 'PICNIC BASKET WICKER LARGE',
 'IVORY KITCHEN SCALES',
 'WHITE WOOD GARDEN PLANT LADDER']

## 구매전 : Top-10 시나리오

In [32]:
def recommend(n=10, acc=False, pred=pred, TFoutputs=TFoutputs):
    item_list = []
    for v in pred.to_list():
        pp = find_item(v, n)
        item_list.append(pp)
    TFoutputs['추천 list'] = item_list
    # user2item['정답 유무'] = [True if label in recommendation(user) else False for user, label in tqdm_notebook(zip(user2item['유저 식별키'], user2item['라벨']), total=len(user2item['유저 식별키']))]
    TFoutputs['정답 유무'] = [True if label in item else False for label, item in tqdm_notebook(zip(TFoutputs['label'], TFoutputs['추천 list']), total=(len(TFoutputs)))]
    # 추천 알고리듬 구매전 Top-10 성능평가
    
    if acc:
        return TFoutputs[TFoutputs['정답 유무']==True].shape[0]/(TFoutputs.shape[0])

    return TFoutputs

In [33]:
recommend(n=5, acc=True)

0.043185169580787865

In [34]:
recommend(n=10, acc=True)

0.06319780914261638

In [35]:
recommend(n=20, acc=True)

0.09163682325679376

In [36]:
recommend()

,train,label,split_train_token,split_label_token,predict_item,filter,추천 list,정답 유무
0,ENAMEL FLOWER JUG CREAM&&DOOR MAT FAIRY CAKE&&...,RED SPOTTY TABLECLOTH,<esp> ENAMEL FLOWER JUG CREAM <esp> DOOR MAT F...,<esp> RED SPOTTY TABLECLOTH <esp>,<esp> ENAMEL BREAD BIN CREAM <esp>,True,"[TEA COSY VINTAGE UNION JACK, ENAMEL MEASURING...",False
1,GAOLERS KEYS DECORATIVE GARDEN&&HAND OPEN SHAP...,SET OF 2 FANCY FONT TEA TOWELS,<esp> GAOLERS KEYS DECORATIVE GARDEN <esp> HAN...,<esp> SET OF 2 FANCY FONT TEA TOWELS <esp>,<esp> PAPER BUNTING RETRO SPOTS <esp>,True,"[PAPER CHAIN KIT RETRO SPOT, STRAWBERRY HONEYC...",False
2,SET OF 9 HEART SHAPED BALLOONS&&SET OF 9 BLACK...,CERAMIC CHERRY CAKE MONEY BANK,<esp> SET OF 9 HEART SHAPED BALLOONS <esp> SET...,<esp> CERAMIC CHERRY CAKE MONEY BANK <esp>,<esp> ASSTD DESIGN BUBBLE GUM RING <esp>,True,"[PIG KEYRING WITH LIGHT & SOUND, DOLLY HONEYCO...",False
3,HANGING HEART ZINC T-LIGHT HOLDER&&OPULENT VEL...,COLOUR GLASS. STAR T-LIGHT HOLDER,<esp> HANGING HEART ZINC T-LIGHT HOLDER <esp> ...,<esp> COLOUR GLASS. STAR T-LIGHT HOLDER <esp>,<esp> HANGING HEART ZINC T-LIGHT HOLDER <esp>,True,"[HANGING RIDGE GLASS T-LIGHT HOLDER, SMALL HAN...",False
4,BAKING SET 9 PIECE RETROSPOT&&PARTY INVITES WO...,PARTY INVITES FOOTBALL,<esp> BAKING SET 9 PIECE RETROSPOT <esp> PARTY...,<esp> PARTY INVITES FOOTBALL <esp>,<esp> PARTY INVITES WOODLAND <esp>,True,"[PARTY INVITES SPACEMAN, PARTY INVITES DINOSAU...",False
...,...,...,...,...,...,...,...,...
14236,ENAMEL MEASURING JUG CREAM&&CREAM HEART CARD H...,IVORY KITCHEN SCALES,<esp> ENAMEL MEASURING JUG CREAM <esp> CREAM H...,<esp> IVORY KITCHEN SCALES <esp>,<esp> DOOR MAT UNION FLAG <esp>,True,"[DOOR MAT NEW ENGLAND, DOOR MAT FANCY FONT HOM...",False
14237,TOILET METAL SIGN&&SAVE THE PLANET MUG&&KINGS ...,POTTERING IN THE SHED METAL SIGN,<esp> TOILET METAL SIGN <esp> SAVE THE PLANET ...,<esp> POTTERING IN THE SHED METAL SIGN <esp>,<esp> BATHROOM METAL SIGN <esp>,True,"[FANNY'S REST STOPMETAL SIGN, YOU'RE CONFUSING...",False
14238,IVORY HANGING DECORATION HEART&&HEART IVORY T...,COLOUR GLASS T-LIGHT HOLDER HANGING,<esp> IVORY HANGING DECORATION HEART <esp> HE...,<esp> COLOUR GLASS T-LIGHT HOLDER HANGING <esp>,<esp> HEART T-LIGHT HOLDER <esp>,True,"[ROSES IN WHITE HEART CANDLE, DECORATION, HANG...",False
14239,LAVENDER SCENTED FABRIC HEART&&PLACE SETTING W...,MURANO STYLE GLASS BRACELET PINK,<esp> LAVENDER SCENTED FABRIC HEART <esp> PLAC...,<esp> MURANO STYLE GLASS BRACELET PINK <esp>,<esp> RED GINGHAM ROSE JEWELLERY BOX <esp>,True,"[ASSORTED ICE CREAM FRIDGE MAGNETS, CAROUSEL D...",False


In [37]:
# item_list = []
# for v in pred.to_list():
#     pp = find_item(v)
#     item_list.append(pp)

# item_list[:2]

In [38]:
# def recommendation(user):
#     n = 10
#     while True:
#         recom_item = set(user2item[user2item['유저 식별키']==find_user(user)]['유저 구매 상품 식별키 묶음'].values[0].split('&&')) - set(user2item[user2item['유저 식별키']==user]['유저 구매 상품 식별키 묶음'].values[0].split('&&'))
#         n += 10
#         if len(recom_item) >= 10:
#             return list(recom_item)[:10]           #Top10
#         if n >= 1000:
#             return list(recom_item)

In [39]:
# # 추천 List 생성

# rec_list = []
# for i in tqdm_notebook(user2item['유저 식별키'].unique()):
#     r_list = list(recommendation(i))
#     rec_list.append(r_list)
# rec_list

# user2rec = pd.DataFrame({'유저 식별키':user2item['유저 식별키'].unique(),
#                          '추천 list':[str(i).replace('[','').replace(']','').replace("'",'').replace(',','&&') 
#                                    for i in rec_list]})
# user2rec

In [40]:
# TFoutputs['추천 list'] = item_list

In [41]:
# # user2item['정답 유무'] = [True if label in recommendation(user) else False for user, label in tqdm_notebook(zip(user2item['유저 식별키'], user2item['라벨']), total=len(user2item['유저 식별키']))]
# TFoutputs['정답 유무'] = [True if label in item else False for label, item in tqdm_notebook(zip(TFoutputs['label'], TFoutputs['추천 list']), total=(len(TFoutputs)))]

In [42]:
# TFoutputs

In [43]:
# # 추천 알고리듬 구매전 Top-10 성능평가

# TFoutputs[TFoutputs['정답 유무']==True].shape[0]/(TFoutputs.shape[0] + falses)

# 구매후(교-차)  : Top-10 시나리오

In [44]:
# def recommendation(user):
#     n = 10
#     while True:
#         recom_item = set(set(user2item[user2item['유저 식별키']==find_user(user)]['유저 구매 상품 식별키 묶음'].values[0].split('@')) & set(find_item(user, n))) - set(user2item[user2item['유저 식별키']==user]['유저 구매 상품 식별키 묶음'].values[0].split('@'))
#         n += 10   
#         if len(recom_item) >= 10:
#             return list(recom_item)[:10]            #Top10
#         if n >= 1000:
#             return list(recom_item)

In [45]:
# # 추천 List 생성

# rec_list = []
# for i in tqdm_notebook(df_f2['고객번호'].unique()):
#     r_list = list(recommendation(i))
#     rec_list.append(r_list)
# rec_list

# user2rec = pd.DataFrame({'유저 식별키':df_f2['고객번호'].unique(),
#                          '추천 list':[str(i).replace('[','').replace(']','').replace("'",'').replace(',','@') 
#                                     for i in rec_list]})
# user2rec

In [46]:
# user2item['정답 유무'] = [True if label in recommendation(user) else False for user, label in tqdm_notebook(zip(user2item['유저 식별키'], user2item['라벨']), total=len(user2item['유저 식별키']))]

In [47]:
# # 추천 알고리듬 구매후(교-차) Top-10 성능평가


# user2item[user2item['정답 유무']==True].shape[0]/user2item.shape[0]*100 #Top10

# 구매후(합-차) : Top-10 시나리오

In [48]:
# def recommendation(user):
#     n = 20
#     while True:
#         recom_item = set(set(user2item[user2item['유저 식별키']==find_user(user)]['유저 구매 상품 식별키 묶음'].values[0].split('@')) | set(find_item(user, n))) - set(user2item[user2item['유저 식별키']==user]['유저 구매 상품 식별키 묶음'].values[0].split('@'))
#         n += 10   
#         if len(recom_item) >= 10:
#             return list(recom_item)[:10]              #Top10
#         if n >= 1000:
#             return list(recom_item)

In [49]:
# # 추천 List 생성

# rec_list = []
# for i in tqdm_notebook(df_f2['고객번호'].unique()):
#     r_list = list(recommendation(i))
#     rec_list.append(r_list)
# rec_list

# user2rec = pd.DataFrame({'유저 식별키':df_f2['고객번호'].unique(),
#                          '추천 list':[str(i).replace('[','').replace(']','').replace("'",'').replace(',','@') 
#                                     for i in rec_list]})
# user2rec

In [50]:
# user2item['정답 유무'] = [True if label in recommendation(user) else False for user, label in tqdm_notebook(zip(user2item['유저 식별키'], user2item['라벨']), total=len(user2item['유저 식별키']))]

In [51]:
# # 추천 알고리듬 구매후(합-차) Top-10 성능평가

# user2item[user2item['정답 유무']==True].shape[0]/user2item.shape[0]*100 #Top10